# Swin Transformers in Pytorch

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models

In [2]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ MPS is available. Using Apple Silicon GPU.")
else:
    device = torch.device("cpu")
    print("❌ MPS not available. Falling back to CPU.")

✅ MPS is available. Using Apple Silicon GPU.


In [3]:
## Now, we import timm, torchvision image models
# !pip install timm
import timm
from timm.loss import LabelSmoothingCrossEntropy # This is better than normal nn.CrossEntropyLoss

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import sys
from tqdm import tqdm
import time
import copy

In [7]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [8]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.Resize((256, 256)),                             # Resize to uniform size
            T.RandomRotation(degrees=10),                     # Small rotation for viewpoint variation
            T.RandomHorizontalFlip(p=0.5),                    # Flip can be safe for lungs (if not labeled left/right)
            T.CenterCrop(224),                                # Crop to match model input
            T.ToTensor(),
            T.Normalize(mean=[0.485], std=[0.229]),           # Normalize for grayscale X-ray images (you may recompute dataset stats)
            T.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3), value='random')  # Simulate occlusions
                ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize((256, 256)),                             # Resize to uniform size
            T.RandomRotation(degrees=10),                     # Small rotation for viewpoint variation
            T.RandomHorizontalFlip(p=0.5),                    # Flip can be safe for lungs (if not labeled left/right)
            T.CenterCrop(224),                                # Crop to match model input
            T.ToTensor(),
            T.Normalize(mean=[0.485], std=[0.229]),           # Normalize for grayscale X-ray images (you may recompute dataset stats)
            T.RandomErasing(p=0.1, scale=(0.02, 0.1), ratio=(0.3, 3.3), value='random')  # Simulate occlusions
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "valid/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "test/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return val_loader, test_loader, len(val_data), len(test_data)

In [9]:
dataset_path = "/Users/zagarsuren/Documents/GitHub/xray-classification/data"

In [10]:
(train_loader, train_data_len) = get_data_loaders(dataset_path, 128, train=True)

In [11]:
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 32, train=False)

In [12]:
classes = get_classes("/Users/zagarsuren/Documents/GitHub/xray-classification/data/train")
print(classes, len(classes))

['Atelectasis', 'Cardiomegaly', 'No Finding', 'Nodule', 'Pneumothorax'] 5


In [13]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [14]:
print(len(train_loader), len(val_loader), len(test_loader))

35 18 18


In [15]:
print(train_data_len, valid_data_len, test_data_len)

4370 545 550


In [16]:
# now, for the model
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("cpu") # torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device

device(type='cpu')

In [17]:
classes

['Atelectasis', 'Cardiomegaly', 'No Finding', 'Nodule', 'Pneumothorax']

In [18]:
HUB_URL = "SharanSMenon/swin-transformer-hub:main"
MODEL_NAME = "swin_tiny_patch4_window7_224"
# check hubconf for more models.
model = torch.hub.load(HUB_URL, MODEL_NAME, pretrained=True) # load from torch hub

Using cache found in /Users/zagarsuren/.cache/torch/hub/SharanSMenon_swin-transformer-hub_main


In [19]:
for param in model.parameters(): #freeze model
    param.requires_grad = False

n_inputs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(n_inputs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(classes))
)
model = model.to(device)
print(model.head)

Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=5, bias=True)
)


In [20]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer = optim.AdamW(model.head.parameters(), lr=0.001)

In [21]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [23]:
import time
import copy
from tqdm import tqdm
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, device, num_epochs=10, class_names=None):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # Lists for tracking metrics
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            all_preds = []
            all_labels = []

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracies.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accuracies.append(epoch_acc.item())

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Print classification report
            if class_names is not None:
                print(f"{phase} Classification Report:")
                print(classification_report(all_labels, all_preds, target_names=class_names))
            else:
                print(f"{phase} Classification Report:")
                print(classification_report(all_labels, all_preds))

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))

    # Plot Loss
    plt.figure()
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss_curve.png')
    plt.close()

    # Plot Accuracy
    plt.figure()
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Val Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('accuracy_curve.png')
    plt.close()

    # Load best model
    model.load_state_dict(best_model_wts)
    return model

In [24]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, dataloaders, dataset_sizes, device, num_epochs=50)

Epoch 0/49
----------


100%|██████████| 35/35 [03:41<00:00,  6.33s/it]


train Loss: 1.5464 Acc: 0.3181
train Classification Report:
              precision    recall  f1-score   support

           0       0.31      0.38      0.34       874
           1       0.32      0.28      0.30       874
           2       0.27      0.24      0.26       874
           3       0.30      0.26      0.28       874
           4       0.38      0.43      0.40       874

    accuracy                           0.32      4370
   macro avg       0.32      0.32      0.32      4370
weighted avg       0.32      0.32      0.32      4370



100%|██████████| 18/18 [00:38<00:00,  2.16s/it]


val Loss: 1.4686 Acc: 0.3835
val Classification Report:
              precision    recall  f1-score   support

           0       0.37      0.39      0.38       109
           1       0.36      0.61      0.46       109
           2       0.17      0.04      0.06       109
           3       0.33      0.45      0.38       109
           4       0.63      0.43      0.51       109

    accuracy                           0.38       545
   macro avg       0.37      0.38      0.36       545
weighted avg       0.37      0.38      0.36       545


Epoch 1/49
----------


...

train Loss: 1.3309 Acc: 0.4886
train Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.50      0.48       874
           1       0.54      0.60      0.57       874
           2       0.42      0.33      0.37       874
           3       0.43      0.40      0.41       874
           4       0.55      0.61      0.58       874

    accuracy                           0.49      4370
   macro avg       0.48      0.49      0.48      4370
weighted avg       0.48      0.49      0.48      4370



100%|██████████| 18/18 [00:39<00:00,  2.21s/it]


val Loss: 1.3818 Acc: 0.4697
val Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.49      0.43       109
           1       0.49      0.57      0.53       109
           2       0.37      0.43      0.40       109
           3       0.46      0.34      0.39       109
           4       0.75      0.52      0.62       109

    accuracy                           0.47       545
   macro avg       0.49      0.47      0.47       545
weighted avg       0.49      0.47      0.47       545


Epoch 27/49
----------


 80%|████████  | 28/35 [02:33<00:38,  5.48s/it]


KeyboardInterrupt: 

## Testing

Ok, now we finished training. Lets run the dataset on the test loader and calculate accuracy

In [ ]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model_ft.eval()

for data, target in tqdm(test_loader):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model_ft(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == 32:
        for i in range(32):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

test_loss = test_loss / test_data_len
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print("Test Accuracy of %5s: %2d%% (%2d/%2d)" % (
            classes[i], 100*class_correct[i]/class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))
print("Test Accuracy of %2d%% (%2d/%2d)" % (
            100*np.sum(class_correct)/np.sum(class_total), np.sum(class_correct), np.sum(class_total)
        ))

In [ ]:
# Save model
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model.cpu(), example)
traced_script_module.save("/Users/zagarsuren/Documents/GitHub/swin-transformer-xray/models/xray_swin_transformer.pt")